Solution to FiveThirtyEight.com Riddler Classic posted Sep 13, 2019

In [1]:
from itertools import permutations
from typing import Sequence

In [2]:
import networkx as nx
import pandas as pd
import requests
from tqdm import tqdm

In [3]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [4]:
resp = requests.get("https://pe.usps.com/text/pub28/28apb.htm", headers=headers)

In [5]:
table = pd.read_html(resp.content)[2]
df = pd.DataFrame(data=table[1:])
df.columns = ["state", "abbreviation"]

In [6]:
g = nx.DiGraph()

In [7]:
g.add_edges_from(
    edge
    for edge in permutations(df.abbreviation, 2)
    if edge[0][1] == edge[1][0]
)

In [14]:
len(g.nodes)

59

In [15]:
len(g.edges)

168

In [8]:
def find_longest_path(source: str, target: str):
    longest = []
    for path in nx.algorithms.simple_paths.all_simple_paths(g, source, target):
        if len(path) > len(longest):
            longest = path
    return longest

In [9]:
def render_path(path: Sequence):
    return "".join(node[0] for node in path) + path[-1][-1]

In [10]:
def calc_answer():
    longest = []
    longest_len = len(longest)
    for pair in tqdm(list(permutations(g.nodes, 2))):
        this_longest_path = find_longest_path(*pair)
        this_longest_len = len(this_longest_path)
        if this_longest_len > longest_len:
            longest = this_longest_path
            longest_len = this_longest_len
    return longest

In [11]:
longest = calc_answer()

100%|██████████| 3422/3422 [1:38:00<00:00,  1.72s/it]  


In [12]:
longest

['FM',
 'MN',
 'NV',
 'VI',
 'ID',
 'DC',
 'CA',
 'AL',
 'LA',
 'AK',
 'KS',
 'SC',
 'CO',
 'OH',
 'HI',
 'IN',
 'NC',
 'CT',
 'TN',
 'NM',
 'MP',
 'PW',
 'WV',
 'VA',
 'AR',
 'RI',
 'IA',
 'AS',
 'SD',
 'DE']

In [13]:
render_path(longest)

'FMNVIDCALAKSCOHINCTNMPWVARIASDE'

I used the Python library networkx to build a directed graph of all the two-letter state and territory abbreviations. Each node was an abbreviation and each edge pointed from a node with a second letter to a node with a matching first letter. The graph had 59 nodes and 168 edges. I then found all simple paths (paths that do not repeat a node) between every possible pair of nodes and kept the longest path. The answer rendered as a string is "FMNVIDCALAKSCOHINCTNMPWVARIASDE". The (admittedly single-threaded) computation took about 1 hour, 38 minutes on my 3.2 GHz Intel Xeon W iMac Pro.